# Oslo bysykkel

In [3]:
import os
import requests
from dotenv import load_dotenv

In [13]:
load_dotenv(os.path.join(os.getcwd(), '.env'))
clientId = os.environ.get('OBC_CLIENT_IDENTIFIER')

# Stations

In [15]:
urlStations = 'https://oslobysykkel.no/api/v1/stations'
headers = {"Client-Identifier": clientId}

In [16]:
stations = requests.get(urlStations, headers=headers)
assert stations.ok
stations = stations.json()['stations']

In [17]:
def searchStations(name):
    for s in stations:
        if s['title'] == name:
            return s

In [18]:
briskeby = searchStations('Briskeby')

In [19]:
briskeby

{'bounds': [{'latitude': 59.920102621492504, 'longitude': 10.717981159687042},
  {'latitude': 59.92013354135258, 'longitude': 10.718072354793549},
  {'latitude': 59.92025049882353, 'longitude': 10.717919468879698},
  {'latitude': 59.920228989434534, 'longitude': 10.717833638191221},
  {'latitude': 59.920102621492504, 'longitude': 10.717981159687042}],
 'center': {'latitude': 59.920187, 'longitude': 10.717955},
 'id': 218,
 'in_service': True,
 'number_of_locks': 15,
 'subtitle': 'ved Uranienborg skole',
 'title': 'Briskeby'}

# Station Availability

In [20]:
urlAvailability = 'https://oslobysykkel.no/api/v1/stations/availability'

In [21]:
avail = requests.get(urlAvailability, headers=headers)
assert avail.ok
avail = avail.json()

In [22]:
avail['refresh_rate'], avail['updated_at']

(10.0, '2017-04-22T13:41:11+00:00')

In [23]:
avail['refresh_rate'], avail['updated_at']

(10.0, '2017-04-22T13:41:11+00:00')

In [24]:
avStat = avail['stations']

In [25]:
def getAvailability(name):
    station = searchStations(name)
    stationId = station['id']
    for s in avStat:
        if s['id'] == stationId:
            return s

In [26]:
getAvailability('Briskeby')

{'availability': {'bikes': 5, 'locks': 7}, 'id': 218}

In [27]:
getAvailability('Fram-gården')

{'availability': {'bikes': 0, 'locks': 27}, 'id': 232}

In [28]:
getAvailability('Gimle Kino')

{'availability': {'bikes': 0, 'locks': 19}, 'id': 219}

In [29]:
getAvailability('Hydroparken')

{'availability': {'bikes': 0, 'locks': 29}, 'id': 261}

# Write to mongodb

Need to create data directory, and start mongod porocess with *mongod --dbpath=[path]*

In [30]:
import pymongo

In [31]:
from pymongo import MongoClient

In [32]:
client = MongoClient('localhost', 27017)

In [33]:
client.address

('localhost', 27017)

In [34]:
db = client.ocbDatabase

In [35]:
collection = db.availability

In [36]:
collection.insert_one(avail).inserted_id

ObjectId('58fb5d855a4bcf4a3e8dc782')

In [37]:
from pprint import pprint

In [38]:
collection.count()

4

In [40]:
client.close()